<center><h1  style="color:white; background-color:#000000; border-radius: 0px; padding:25px;"> Theoretical study of SMOTE </h1></center>

This notebook reproduces the numerical experiments.

<ins>Experimented Datasets: </ins>
- [Phoneme](#Phoneme)
- Pima
- Abalone
- Haberman
- Yeast
- Wine
- Vehicle
- Bresat cancer Wisconsin
- Ionosphere
- CreditCard
- MagicTel
- California
- House_16H

ALl the sections of this notebook are indepent. user should run the following two introduction code chunk. Then one can choose a given data set section and run exclusivey the chunks code of the selected section. Fot the subsampled data sets version, all the seed and are given and the subsampling are each time included in the given section. However, the protocol is always pre-filed with the smalles (most undersampled) version of the dat set. On can try other subsampled data sets by changing the protocol data sets used.


Tuning dpeth part is at the end of tghe notebook. The used depth for each data set are given. user should fill the protocol with the desired data sets and their associated depth dictionnary.



In [ ]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from pathlib import Path


from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler,NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE


from oversampling_strategies.oversampling_strategies import MGS,MGS2, NoSampling, CVSmoteModel

from validation.classif_experiments import run_eval, subsample_to_ratio_indices,read_subsampling_indices, compute_metrics_several_protocols

In [ ]:
output_dir_path =  "../saved_experiments" ## Fill it
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
output_dir_path

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;" name='Phoneme'> Phoneme</h1></center>

## <a name='Phoneme'></a>

## Run protocol :

In [ ]:
from data.data import load_phoneme_data
X_phoneme, y_phoneme = load_phoneme_data()

In [ ]:
indices_kept_20 = subsample_to_ratio_indices(X=X_phoneme,y=y_phoneme,ratio=0.2,seed_sub=11,
                                           output_dir_subsampling=output_dir_path,
                                           name_subsampling_file='phoneme_sub_original_to_20')

indices_kept_10 = subsample_to_ratio_indices(
    X=X_phoneme,y=y_phoneme,ratio=0.1,seed_sub=9,
    output_dir_subsampling=output_dir_path,
    name_subsampling_file='phoneme_sub_20_to_10',has_previous_under_sampling=True,
    previous_under_sampling=indices_kept_20)

indices_kept_1 = subsample_to_ratio_indices(X=X_phoneme,y=y_phoneme,ratio=0.01,seed_sub=5,
    output_dir_subsampling=output_dir_path,
    name_subsampling_file='phoneme_sub_10_to_1',has_previous_under_sampling=True,
    previous_under_sampling=indices_kept_10)

In [ ]:
X_phoneme_1,y_phoneme_1 = read_subsampling_indices(X=X_phoneme,y=y_phoneme, dir_subsampling=output_dir_path, name_subsampling_file='phoneme_sub_10_to_1', get_indexes=False)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('MGS',MGS(K=6),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_phoneme_depthNone_test'
for i in range(20):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_phoneme_1,y=y_phoneme_1,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute metrics :

In [ ]:
list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_phoneme,df_final_std_phoneme = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_phoneme_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=20)
    

df_final_mean_phoneme.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Pima </h1></center>

## Run protocol :

In [ ]:
from data.data import load_pima_data
X_pima, y_pima = load_pima_data()

In [ ]:
indices_kept_20 = subsample_to_ratio_indices(X=X_pima,y=y_pima,ratio=0.2,seed_sub=15,
                                           output_dir_subsampling=output_dir_path,
                                           name_subsampling_file='pima_sub_original_to_20')


In [ ]:
X_pima_20,y_pima_20 = read_subsampling_indices(X=X_pima,y=y_pima, dir_subsampling=output_dir_path, name_subsampling_file='pima_sub_original_to_20', get_indexes=False)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=9,n_points=9,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_pima_depthNone_test'
for i in range(20):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_pima_20,y=y_pima_20,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute metrics :

In [ ]:
list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_pima,df_final_std_pima = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_pima_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=20)

df_final_mean_pima.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Abalone </h1></center>

## Run protocol :

In [ ]:
from data.data import load_abalone_data
X_abalone, y_abalone = load_abalone_data()

In [ ]:
from collections import Counter
Counter(y_abalone)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=9,n_points=9,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_abalone_depthNone_test'
for i in range(100):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_abalone,y=y_abalone,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute metrics :

In [ ]:
list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_abalone,df_final_std_abalone = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_abalone_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=100)

df_final_mean_abalone.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Haberman </h1></center>

## Run protocol :

In [ ]:
from data.data import load_haberman_data
X_haberman,y_haberman = load_haberman_data()

In [ ]:
indices_kept_10 = subsample_to_ratio_indices(X=X_haberman,y=y_haberman,ratio=0.1,seed_sub=11,
                                           output_dir_subsampling=output_dir_path,
                                           name_subsampling_file='haberman_sub_original_to_20')
X_haberman_10,y_haberman_10 = read_subsampling_indices(X=X_haberman,y=y_haberman, dir_subsampling=output_dir_path, name_subsampling_file='haberman_sub_original_to_20', get_indexes=False)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_depth = [5,15,19,24,29,32]
grid_RF = GridSearchCV(estimator=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=10),
             param_grid={'max_depth': list_depth }, scoring='roc_auc')

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=5,n_points=5,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_haberman_depthNone_test'
for i in range(100):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_haberman_10,y=y_haberman_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute metrics :

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from validation.classif_experiments import compute_metrics_several_protocols

list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_haberman,df_final_std_haberman = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_haberman_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=100)

df_final_mean_haberman.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Yeast </h1></center>

In [ ]:
from data.data import load_yeast_data
X_yeast,y_yeast = load_yeast_data()

In [ ]:
indices_kept_10 = subsample_to_ratio_indices(X=X_yeast,y=y_yeast,ratio=0.01,seed_sub=15,
                                           output_dir_subsampling=output_dir_path,
                                           name_subsampling_file='yeast_sub_original_to_1')

X_yeast_1,y_yeast_1 = read_subsampling_indices(X=X_yeast,y=y_yeast, dir_subsampling=output_dir_path, name_subsampling_file='yeast_sub_original_to_1', get_indexes=False)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=7,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=9,n_points=9,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_yeast_depthNone_test'
for i in range(100):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_yeast_1,y=y_yeast_1,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute Metrics :

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from validation.classif_experiments import compute_metrics_several_protocols

list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_haberman,df_final_std_haberman = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_yeast_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=100)

df_final_mean_haberman.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Wine </h1></center>

Wine quality datas et : class 5 and 6 vs 8. Do not forget to install ucimlrepo. # pip install ucimlrepo

In [ ]:
from data.data import load_wine_data
X_wine,y_wine = load_wine_data()

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=12,n_points=12,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_wine_depthNone_test'
for i in range(100):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_wine,y=y_wine,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute Metrics :

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from validation.classif_experiments import compute_metrics_several_protocols

list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_haberman,df_final_std_haberman = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_wine_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=100)

df_final_mean_haberman.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Vehicle </h1></center>

## Run protocol :

In [ ]:
from data.data import load_vehicle_data
X_vehicle, y_vehicle = load_vehicle_data()

In [ ]:
indices_kept_10 = subsample_to_ratio_indices(X=X_vehicle,y=y_vehicle,ratio=0.1,seed_sub=11,
    output_dir_subsampling=output_dir_path,
    name_subsampling_file='vehicle_sub_original_to_10')

X_vehicle_10, y_vehicle_10 = read_subsampling_indices(X=X_vehicle,y=y_vehicle, dir_subsampling=output_dir_path, name_subsampling_file='vehicle_sub_original_to_10', get_indexes=False)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=19,n_points=19,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_vehicle_depthNone_test'
for i in range(100):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_vehicle_10,y=y_vehicle_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute Metrics :

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from validation.classif_experiments import compute_metrics_several_protocols

list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_vehicle,df_final_std_vehicle = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_vehicle_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=100)

df_final_mean_vehicle.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Breast cancer Wisconsin </h1></center>

## Run protocol :

In [ ]:
from data.data import load_breastcancer_data
X_bcw, y_bcw = load_breastcancer_data()

In [ ]:
indices_kept_20 = subsample_to_ratio_indices(X=X_bcw,y=y_bcw,ratio=0.2,seed_sub=11,
                                           output_dir_subsampling=output_dir_path,
                                           name_subsampling_file='bcw_sub_original_to_20')

indices_kept_10 = subsample_to_ratio_indices(
    X=X_bcw,y=y_bcw,ratio=0.1,seed_sub=9,
    output_dir_subsampling=output_dir_path,
    name_subsampling_file='bcw_sub_20_to_10',has_previous_under_sampling=True,
    previous_under_sampling=indices_kept_20)

X_bcw_10, y_bcw_10 = read_subsampling_indices(X=X_bcw,y=y_bcw, dir_subsampling=output_dir_path, name_subsampling_file='bcw_sub_20_to_10', get_indexes=False)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=10,n_points=10,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_bcw_depthNone_test'
for i in range(100):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_vehicle_10,y=y_vehicle_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute Metrics :

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from validation.classif_experiments import compute_metrics_several_protocols

list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_haberman,df_final_std_haberman = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_bcw_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=100)

df_final_mean_haberman.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Ionosphere </h1></center>

## Run protocol :

In [ ]:
from data.data import load_ionosphere_data
X_ionosphere, y_ionosphere = load_ionosphere_data()

In [ ]:
indices_kept_20 = subsample_to_ratio_indices(X=X_ionosphere,y=y_ionosphere,ratio=0.2,seed_sub=11,
                                           output_dir_subsampling=output_dir_path,
                                           name_subsampling_file='ionosphere_sub_original_to_20')

indices_kept_10 = subsample_to_ratio_indices(
    X=X_ionosphere,y=y_ionosphere,ratio=0.1,seed_sub=9,
    output_dir_subsampling=output_dir_path,
    name_subsampling_file='ionosphere_sub_20_to_10',has_previous_under_sampling=True,
    previous_under_sampling=indices_kept_20)

X_ionosphere_10, y_ionosphere_10 = read_subsampling_indices(X=X_ionosphere,y=y_ionosphere, dir_subsampling=output_dir_path, name_subsampling_file='ionosphere_sub_20_to_10', get_indexes=False)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=5,n_points=5,llambda=1.0),{},rf_model), # Not possible for Inosphere 10% to have MGS K=33 (wo we take K=5). Inosphere and Ionopshere 20% were performed with K=33, change it if they are used
]

In [ ]:
init_name_file = '2024-02-14-RF100_ionosphere_depthNone_test'
for i in range(100):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_ionosphere_10,y=y_ionosphere_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute Metrics :

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from validation.classif_experiments import compute_metrics_several_protocols

list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_haberman,df_final_std_haberman = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_ionosphere_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=100)

df_final_mean_haberman.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> CreditCard </h1></center>

## Run protocol :

In [ ]:
from data.data import load_credit_data
X_credit, y_credit, meta_df_credit = load_credit_data()

In [ ]:
from validation.classif_experiments import PaperTimeSeriesSplitWithGroupOut
splitter_PTSSG = PaperTimeSeriesSplitWithGroupOut(n_splits=5, starting_split=0,meta_df=meta_df_credit,col_name_id='Time')

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=30,n_points=30,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_credit_depthNone_test'
for i in range(1):
# We do not change the splitter at each iteration here due to the temporal split
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_credit,y=y_credit,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_PTSSG)

## Compute Metrics :

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from validation.classif_experiments import compute_metrics_several_protocols

list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_credit,df_final_std_credit = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_credit_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=100)

df_final_mean_credit.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> MagicTel </h1></center>

## Run protocol : 


In [ ]:
from data.data import load_magictel_data
X_magic,y_magic = load_magictel_data()

In [ ]:
indices_kept_20 = subsample_to_ratio_indices(X=X_magic,y=y_magic,ratio=0.2,seed_sub=11,
                                           output_dir_subsampling=output_dir_path,
                                           name_subsampling_file='magicTel_sub_original_to_20')

In [ ]:
X_magic_20,y_magic_20 = read_subsampling_indices(X=X_magic,y=y_magic, dir_subsampling=output_dir_path, name_subsampling_file='magicTel_sub_original_to_20', get_indexes=False)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=100)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=11,n_points=11,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_magicTel_depthNone_test'
for i in range(100):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_magic_20,y=y_magic_20,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute Metrics :

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from validation.classif_experiments import compute_metrics_several_protocols

list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_haberman,df_final_std_haberman = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_magicTel_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=10)

df_final_mean_haberman.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> California </h1></center>

In [ ]:
from data.data import load_california_data
X_california,y_california = load_california_data()

In [ ]:
indices_kept_20 = subsample_to_ratio_indices(X=X_california,y=y_california,ratio=0.2,seed_sub=11,
                                           output_dir_subsampling=output_dir_path,
                                           name_subsampling_file='california_sub_original_to_20')

indices_kept_10 = subsample_to_ratio_indices(
    X=X_california,y=y_california,ratio=0.1,seed_sub=9,
    output_dir_subsampling=output_dir_path,
    name_subsampling_file='california_sub_20_to_10',has_previous_under_sampling=True,
    previous_under_sampling=indices_kept_20)

indices_kept_1 = subsample_to_ratio_indices(X=X_california,y=y_california,ratio=0.01,seed_sub=5,
    output_dir_subsampling=output_dir_path,
    name_subsampling_file='california_sub_10_to_1',has_previous_under_sampling=True,
    previous_under_sampling=indices_kept_10)

In [ ]:
X_california_1,y_california_1 = read_subsampling_indices(X=X_california,y=y_california, dir_subsampling=output_dir_path, name_subsampling_file='california_sub_10_to_1', get_indexes=False)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=9,n_points=9,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_california_depthNone_test'
for i in range(100):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_california_1,y=y_california_1,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute Metrics :

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from validation.classif_experiments import compute_metrics_several_protocols

list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_haberman,df_final_std_haberman = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_california_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=100)

df_final_mean_haberman.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> House_16H </h1></center>

In [ ]:
from data.data import load_house_data
X_house,y_house = load_house_data()

In [ ]:
indices_kept_20 = subsample_to_ratio_indices(X=X_house,y=y_house,ratio=0.2,seed_sub=11,
                                           output_dir_subsampling=output_dir_path,
                                           name_subsampling_file='house16_sub_original_to_20')

indices_kept_10 = subsample_to_ratio_indices(
    X=X_house,y=y_house,ratio=0.1,seed_sub=9,
    output_dir_subsampling=output_dir_path,
    name_subsampling_file='house16_sub_20_to_10',has_previous_under_sampling=True,
    previous_under_sampling=indices_kept_20)

indices_kept_1 = subsample_to_ratio_indices(X=X_house,y=y_house,ratio=0.01,seed_sub=5,
    output_dir_subsampling=output_dir_path,
    name_subsampling_file='house16_sub_10_to_1',has_previous_under_sampling=True,
    previous_under_sampling=indices_kept_10)

In [ ]:
X_house_1,y_house_1 = read_subsampling_indices(X=X_house,y=y_house, dir_subsampling=output_dir_path, name_subsampling_file='house16_sub_10_to_1', get_indexes=False)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_rf_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
CVSmoteRF = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

list_oversampling_and_params = [
    ('None', NoSampling(), {}, rf_model),
    ('c_weight', NoSampling(), {}, balanced_rf_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, rf_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, rf_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, rf_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, rf_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, rf_model),
    ('SMOTE',SMOTE(sampling_strategy="minority"),{},rf_model),
    ('cv_smote', NoSampling(),{}, CVSmoteRF),
    ('MGS',MGS(K=17,n_points=17,llambda=1.0),{},rf_model),
]

In [ ]:
init_name_file = '2024-02-14-RF100_house16_depthNone_test'
for i in range(100):
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_house_1,y=y_house_1,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute Metrics :

In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from validation.classif_experiments import compute_metrics_several_protocols

list_metric = [
    (precision_score,'precision','pred'),
    (recall_score,'recall','pred'),
    (roc_auc_score,'roc_auc','proba')
]

In [ ]:
df_final_mean_haberman,df_final_std_haberman = compute_metrics_several_protocols(
    output_dir=output_dir_path,
    init_name_file='2024-02-14-RF100_house16_depthNone_test',
    list_metric=list_metric,
    bool_roc_auc_only=True,n_iter=100)

df_final_mean_haberman.style.format(precision=3)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Tuning depth </h1></center>

For tunnig depth, for each data sets, we look at the mean depth of the forests for a given strategy (computed when the protocol is running and saved in a file). Then we applied a gridsearch with a grid for max_depth parameter based on the maximum value previously selected.
The maximum value dictionnary of depth for ecah highly imbalanced dat are given below.

In [ ]:
from validation.distances_experiments import depth_func_linspace

In [ ]:
max_depth_dict_by_strat_pima_20 = {'none':13,
                           'smote':15,
                           'MGS':15,
                           'cweight':14,
                           'BS1':15,
                           'BS2':16,
                           'NM1':11,
                           'RUS':10,
                           'ROS':14,
                           'cvsmote':15,
                          }

In [ ]:
max_depth_dict_by_strat_abalone = {'none': 13,
                           'smote':22,
                           'MGS':22,
                           'cweight':18,
                           'BS1': 20,
                           'BS2': 21,
                           'NM1':7,
                           'RUS':7,
                           'ROS':18,
                           'cvsmote':23,
                          }

In [ ]:
max_depth_dict_by_strat_phoneme_1 = {'none': 11,
                           'smote':20,
                           'MGS':19,
                           'cweight':15,
                           'BS1': 16,
                           'BS2': 19,
                           'NM1':6,
                           'RUS':6,
                           'ROS':17,
                           'cvsmote':21,
                          }

In [ ]:
max_depth_dict_by_strat_yeast_1 = {'none': 9,
                           'smote':14,
                           'MGS':10,
                           'cweight':11,
                           'BS1': 13,
                           'BS2': 15,
                           'NM1':5,
                           'RUS':4,
                           'ROS':13,
                           'cvsmote':14,
                          }

In [ ]:
max_depth_dict_by_strat_haberman_10 = {'none': 10,
                           'smote':14,
                           'MGS':14,
                           'cweight':10,
                           'BS1': 13,
                           'BS2': 13,
                           'NM1':7,
                           'RUS':7,
                           'ROS':11,
                           'cvsmote':13,
                          }

In [ ]:
max_depth_dict_by_strat_wine = {'none': 16,
                           'smote':25,
                           'MGS':23,
                           'cweight':24,
                           'BS1': 25,
                           'BS2': 26,
                           'NM1':14,
                           'RUS':10,
                           'ROS':24,
                           'cvsmote':26,
                          }

In [ ]:
max_depth_dict_by_strat_magic_20 = {'none':23,
                           'smote':30,
                           'MGS':26,
                           'cweight':29,
                           'BS1':31,
                           'BS2':33,
                           'NM1':21,
                           'RUS':20,
                           'ROS':29,
                           'cvsmote':30,
                          }

In [ ]:
max_depth_dict_by_strat_creditcard = {'none': 18,
                 'smote': 29,
                 'MGS':36,
                 'cweight':20,
                 'BS1': 23,
                 'BS2':29,
                 'NM1':8,
                 'RUS':10,
                 'ROS':22,
                 'cvsmote':29,}

In [ ]:
max_depth_dict_by_strat_california_1 = {'none':17,
                           'smote':26,
                           'MGS':26,
                           'cweight':21,
                           'BS1':23,
                           'BS2':25,
                           'NM1':9,
                           'RUS':10,
                           'ROS':24,
                           'cvsmote':27,
                          }

In [ ]:
max_depth_dict_by_strat_vehicle_original = {'none':10,
                                            'smote':12,
                                            'MGS':11,
                                            'cweight':12,
                                            'BS1':12,
                                            'BS2':12,
                                            'NM1':9,
                                            'RUS':9,
                                            'ROS':12,
                                            'cvsmote':12,
                                        }

In [ ]:
max_depth_dict_by_strat_vehicle_10 = {'none':9,
                           'smote':11,
                           'MGS':11,
                           'cweight':11,
                           'BS1':11,
                           'BS2':11,
                           'NM1':7,
                           'RUS':6,
                           'ROS':12,
                           'cvsmote':11,
                          }

In [ ]:
max_depth_dict_by_strat_bcw_original = {'none':8,
                           'smote':8,
                           'MGS':8,
                           'cweight':8,
                           'BS1':9,
                           'BS2':9,
                           'NM1':7,
                           'RUS':7,
                           'ROS':8,
                           'cvsmote':8,
                          }

In [ ]:
max_depth_dict_by_strat_bcw_20 = {'none':6,
                           'smote':7,
                           'MGS':7,
                           'cweight':7,
                           'BS1':7,
                           'BS2':8,
                           'NM1':6,
                           'RUS':4,
                           'ROS':7,
                           'cvsmote':7,
                          }

In [ ]:
max_depth_dict_by_strat_bcw_10 = {'none':4,
                           'smote':6,
                           'MGS':6,
                           'cweight':6,
                           'BS1':7,
                           'BS2':7,
                           'NM1':4,
                           'RUS':3,
                           'ROS':7,
                           'cvsmote':6,
                          }

In [ ]:
max_depth_dict_by_strat_ionosphere_original = {'none':10,
                           'smote':10,
                           'MGS':10,
                           'cweight':9,
                           'BS1':10,
                           'BS2':11,
                           'NM1':7,
                           'RUS':8,
                           'ROS':9,
                           'cvsmote':10,
                          }

In [ ]:
max_depth_dict_by_strat_ionosphere_20 = {'none':7,
                           'smote':8,
                           'MGS':8,
                           'cweight':6,
                           'BS1':7,
                           'BS2':8,
                           'NM1':4,
                           'RUS':4,
                           'ROS':7,
                           'cvsmote':8,
                          }

In [ ]:
max_depth_dict_by_strat_ionosphere_10 = {'none':4,
                           'smote':7,
                           'MGS':7,
                           'cweight':4,
                           'BS1':6,
                           'BS2':7,
                           'NM1':3,
                           'RUS':3,
                           'ROS':6,
                           'cvsmote':7,
                          }

User should selec a higly imbalanced data set max depth dictionnary. Then, change the K value of MGS (K=dimension of the selected data set). Finnaly fill the data in run_eval function before launching the protocol.

In [ ]:
max_depth_dict_by_strat = #Filll it

for strat in max_depth_dict_by_strat:
    max_depth = max_depth_dict_by_strat[strat]
    list_depth = depth_func_linspace(min_value=5,max_value=max_depth,size=8)
    ############
    GRF = GridSearchCV(estimator=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=10),
                       param_grid={'max_depth': list_depth }, scoring='roc_auc')
    GRF2 = GridSearchCV(estimator=RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=10),
                        param_grid={'max_depth': list_depth }, scoring='roc_auc')
    
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=None)
    CVSmoteGRF = CVSmoteModel(splitter=splitter_stratified,
                             model=GridSearchCV(estimator=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=10),
                                                param_grid={'max_depth': list_depth}, scoring='roc_auc'),
                                                list_k_max=15,list_k_step=1)
    
    list_oversampling_dict_by_strat = {'none': [('None', NoSampling(), {}, GRF),],
                                       'smote': [('SMOTE',SMOTE(sampling_strategy="minority"),{},GRF),],
                                       'MGS':[('MGS',MGS(K=9,n_points=9,llambda=1.0),{},GRF)], # Adapt the dimension : K=n_points=dimesnion+1
                                       'cweight':[('c_weight', NoSampling(), {}, GRF2),],
                                       'BS1': [('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1"),{}, GRF),],
                                       'BS2': [('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2"),{}, GRF),],
                                       'NM1':[('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, GRF),],
                                       'RUS':[('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False),{}, GRF),],
                                       'ROS':[('ROS',  RandomOverSampler(sampling_strategy="minority"),{}, GRF),],
                                       'cvsmote': [('cv_smote', NoSampling(),{}, CVSmoteGRF),]}
    list_oversampling_and_params = list_oversampling_dict_by_strat[strat]
    #############
    output_dir=os.path.join(output_dir_path,strat,'RF_100')
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    init_name_file = '2024-02-14-RF100_tuned_depthNone_'
    for i in range(100):
        splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
        name_file = init_name_file + str(i) +'.npy'
        run_eval(output_dir=output_dir,name_file=name_file,X=,y=, # Fill data
                 list_oversampling_and_params=list_oversampling_and_params,
                 splitter=splitter_stratified,to_standard_scale=True)

In [ ]:
list_name_strat = ['none','cweight','RUS','ROS','NM1','BS1','BS2','smote','cvsmote','MGS']
for strat in list_name_strat:
    output_dir=output_dir=os.path.join(output_dir_path,strat,'RF_100')
    init_name_file = '2024-02-14-RF100_tuned_depthNone_'

    df_final_mean_abalone,df_final_std_abalone = compute_metrics_several_protocols(output_dir=output_dir,init_name_file=init_name_file,
                                                                                   list_metric=list_metric,bool_roc_auc_only=False,n_iter=20)
    mean_roc = round(df_final_mean_abalone.to_numpy()[0,0],3)
    srtd_roc = round(df_final_std_abalone.to_numpy()[0,0],3)
    print("########"*5)
    print('****** Strategy : ',strat, '******')
    print("ROC AUC mean : ",mean_roc)
    print("ROC AUC std : ",srtd_roc)

<center><h1  style="color:white; background-color:#008b96; border-radius: 10px; padding:15px;"> END </h1></center>